# 1. Configurations

## 1.1 Install Packages

In [ ]:
!pip install colab_ssh

from colab_ssh import launch_ssh_cloudflared
launch_ssh_cloudflared(password='test')

from google.colab import drive
drive.mount('/mnt')

!ln -s /mnt/MyDrive/court-decision-prediction /root
%cd /root/court-decision-prediction

In [ ]:
!pip install transformers==4.30.2 loralib==0.1.1 sentencepiece==0.1.99 livelossplot==0.5.5

In [ ]:
!cp /mnt/MyDrive/modeling_llama.py /usr/local/lib/python3.10/dist-packages/transformers/models/llama/modeling_llama.py

Modify file for lora: `/usr/local/lib/python3.10/dist-packages/transformers/models/llama/modeling_llama.py`

https://github.com/microsoft/LoRA#quickstart
```
# ===== Before =====
# layer = nn.Linear(in_features, out_features)

# ===== After ======
import loralib as lora
# Add a pair of low-rank adaptation matrices with rank r=16
layer = lora.Linear(in_features, out_features, r=16)
```

In [ ]:
import os
from os.path import join, dirname
import random
import warnings

import numpy as np
import pandas as pd

import torch
from torch import nn

import loralib as lora

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
warnings.filterwarnings('ignore')

## 1.2 Random Seed

In [ ]:
def seed_everything(seed: int=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

## 1.3 PATH

In [ ]:
class PATH:
    root   = '/root/court-decision-prediction'
    yaml   = join(root, 'court_decision_prediction/configs.yaml')
    data   = join(root, 'data/open')
    train  = join(data, 'train.csv')
    test   = join(data, 'test.csv')
    sample = join(data, 'sample_submission.csv')
    submit = join(root, 'submission')

# 2. Load Dataset

## 2.1 Load Raw Data

In [ ]:
train_full_data = pd.read_csv(PATH.train)
test_data       = pd.read_csv(PATH.test)
target          = 'first_party_winner'

## 2.2 Preprocessing
1. Group party into `united states`, `states`, `group1`, `group2`, `indiv`
2. Handle data imbalance
    - Stratified undersampling(`first_party_grp`)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample


def preprocess(data, target):
    data = append_party_group(data)
    if target in data:
        data = undersampling(data, target)
    return data


def append_party_group(data):
    data = data.copy()
    states = list(map(str.lower,
                      ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
                       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
                       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
                       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
                       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']))
    cities = ['new york', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
              'dallas', 'san francisco', 'oakland', 'austin', 'jacksonville', 'san jose', 'california', 'indianapolis',
              'seattle', 'denver', 'washington', 'boston', 'el paso', 'nashville', 'detroit', 'oklahoma', 'portland',
              'las vegas', 'memphis', 'louisville', 'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'mesa',
              'sacramento', 'atlanta', 'kansas city', 'colorado springs', 'omaha', 'raleigh', 'miami', 'long beach',
              'virginia beach', 'oakland', 'minneapolis', 'tulsa', 'tampa', 'arlington', 'new orleans', 'wichita',
              'cleveland', 'bakersfield', 'aurora', 'anaheim', 'honolulu', 'santa ana', 'riverside', 'corpus christi',
              'lexington', 'stockton', 'anchorage', 'st. paul', 'newark', 'buffalo', 'plano', 'henderson', 'fort wayne',
              'greensboro', 'lincoln', 'glendale', 'chandler', 'st. petersburg', 'jersey city', 'scottsdale', 'norfolk',
              'madison', 'orlando', 'birmingham', 'baton rouge', 'durham', 'laredo', 'garland', 'chula vista',
              'riverside', 'hialeah', 'lubbock', 'reno', 'north las vegas', 'akron', 'gilbert', 'rochester', 'boise',
              'spokane']
    def generate_fn(col):
        def fn(row):
            party = row[col].lower()

            keywords1 = ['united states', 'federal', 'commision', 'commodity', 'national']
            keywords2 = states + cities + ['school board', 'city', 'republic', 'region', 'district', 'county']
            keywords3 = ['corporation', 'inc', 'company', 'bank', 'association', 'llc', 'co.', 'hospital', 'usa', 'school', 'group', 'office', 'department']
            keywords4 = [',', 'et al']

            for grp, keywords in zip(['united states', 'states', 'group1', 'group2'],
                                    [keywords1, keywords2, keywords3, keywords4]):
                for key in keywords:
                    if key in party:
                        return grp
            return 'indiv'
        return fn

    data['first_party_grp']  = data.apply(generate_fn('first_party'), axis=1)
    data['second_party_grp'] = data.apply(generate_fn('second_party'), axis=1)
    return data

def undersampling(data, target):
    data_zero_target = data[data[target] == 0]
    data_one_target  = data[data[target] == 1]
    first_party_grp_int = LabelEncoder().fit_transform(data_one_target['first_party_grp'])
    data_one_target_sample = resample(data_one_target, replace=False, n_samples=len(data_zero_target), stratify=first_party_grp_int)
    return pd.concat([data_zero_target, data_one_target_sample], ignore_index=True)

In [ ]:
train_full_data_pp = preprocess(train_full_data, target)
test_data_pp       = preprocess(test_data, target)

In [ ]:
def plot_party(ax1, ax2, f):
    train_full_data_pp[f].value_counts().head(10).plot.bar(ax=ax1),        ax1.set_title(f"{f} (# unique: {train_full_data_pp[f].nunique()})")
    train_full_data_pp[f'{f}_grp'].value_counts().plot.bar(ax=ax2, rot=0), ax2.set_title(f"{f}_grp (# unique: {train_full_data_pp[f+'_grp'].nunique()})")

for f in ('first_party', 'second_party'):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 3))
    plot_party(ax1, ax2, f)

In [ ]:
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 3))
train_full_data[target].plot.hist(ax=ax1),    ax1.set_title(f"{target} (before undersampling)")
train_full_data_pp[target].plot.hist(ax=ax2), ax2.set_title(f"{target} (after undersampling)");

## 2.3 Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_full_data_pp, test_size=0.3, stratify=train_full_data_pp[target])
len(train_data), len(val_data)

# 3. Modeling

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id  = "lmsys/vicuna-7b-v1.3"
# model_id  = "openlm-research/open_llama_3b"
# model_id  = "lmsys/fastchat-t5-3b-v1.0"

model     = LlamaForCausalLM.from_pretrained(model_id).to(torch.bfloat16)
tokenizer = LlamaTokenizer.from_pretrained(model_id)

## 3.1 Test Model

In [ ]:
# prompt = f"""Question: What is 1+1?
# Answer: """
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
# with torch.no_grad():
#     outputs = model.generate(input_ids=input_ids, max_new_tokens=8)
# response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response)

## 3.2 Test Tokenizer

In [ ]:
# print("O:", tokenizer.decode([438, 29949]))   # 29949: _O
# print("X:", tokenizer.decode([1060, 29990]))  # 29990: _X
# print("Start token:", tokenizer.decode([1]))

## 3.3 Modify Model

In [ ]:
# Lora setting
model.requires_grad = False
lora.mark_only_lora_as_trainable(model)

# Change trainable head
# model.lm_head = nn.Linear(3200, 1)  # 4096: hidden size, 1: binary classification (w.o. sigmoid)
model.lm_head = nn.Linear(4096, 1)  # 4096: hidden size, 1: binary classification (w.o. sigmoid)
model.lm_head.requires_grad = True
model = model.to(torch.bfloat16).cuda()

In [ ]:
model

# 4. Train

In [ ]:
from torch.utils.data import Dataset, DataLoader


def get_prompt(first_party, second_party, facts):
    return f"""A chat between a curious user and an artificial intelligence assistant.
The assistant gives helpful and concise answers to the user's questions.

USER:
- first_party: {first_party}
- second_party: {second_party}
- facts:
{facts[:1000]}

- Question: I want to know whether the first_party can win the case. Tell me just the answer with O or X, without any detailed reasons.

ASSISTANT:
- Answer: """

# def get_prompt(first_party, second_party, facts):
#     return f"""
# USER:
# - first_party: {first_party}
# - second_party: {second_party}
# - facts:
# {facts[:1000]}
# - Question: I want to know whether the first_party can win the case. Tell me just the answer with O or X, without any detailed reasons.

# ASSISTANT:
# - Answer: """


# def decode(output):
#     return tokenizer.decode(output[0], skip_special_tokens=True)

class CustomDataset(Dataset):
    def __init__(self, data, target, tokenizer):
        self.data      = data
        self.target    = target

        tokenizer.pad_token = tokenizer.eos_token  # use default pad token
        self.input_ids = self._get_input_ids(data, tokenizer)
        self.labels    = self._get_labels(data)

    def _get_input_ids(self, data, tokenzier):
        prompts = []
        for idx_row in range(len(data)):
            row = self.data.iloc[idx_row]
            prompt = get_prompt(row['first_party'], row['second_party'], row['facts'])
            prompts.append(prompt)
        return tokenizer(prompts, return_tensors='pt', padding='longest').input_ids.cuda()

    def _get_labels(self, data):
        if self.target in data:
            return len(data)*[None]

        labels = []
        for idx_row in range(len(data)):
            row = data.iloc[idx_row]
            label = row[self.target]
            labels.append(label)
        return torch.tensor(labels, dtype=torch.bfloat16).cuda()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input = self.input_ids[idx]
        label = self.labels[idx]
        return input, label


train_ds = CustomDataset(train_data, target, tokenizer)
val_ds   = CustomDataset(val_data, target, tokenizer)

train_dl = DataLoader(train_ds, batch_size=1, shuffle=True)
val_dl   = DataLoader(val_ds,   batch_size=1, shuffle=False)

In [ ]:
from livelossplot import PlotLosses
from tqdm import trange, tqdm


def compute_loss_acc(X, y):
    output = model(X)
    logit  = output[0][0, -1]  # output of last sequence
    loss   = loss_fn(logit, y)
    acc    = (logit > 0) == y
    return loss, acc

def train_step(X, y):
    loss, acc = compute_loss_acc(X, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item(), acc.item()

def val_step(X, y):
    loss, acc = compute_loss_acc(X, y)
    return loss.item(), acc.item()


loss_fn   = nn.BCEWithLogitsLoss().cuda()
optimizer = torch.optim.Adam(model.parameters())

liveloss = PlotLosses()
history  = pd.DataFrame(columns=['loss', 'acc', 'val_loss', 'val_acc'], index=pd.Index([], name='Epoch'))
n_epochs = 10
for epoch in trange(1, n_epochs+1):
    avg_loss, avg_acc = [], []
    avg_val_loss, avg_val_acc = [], []

    for X, y in tqdm(train_dl):
        loss, acc = train_step(X, y)
        avg_loss.append(loss), avg_acc.append(acc)

    with torch.no_grad():
        for X, y in tqdm(val_dl):
            val_loss, val_acc = val_step(X, y)
            avg_val_loss.append(val_loss), avg_val_acc.append(val_acc)

    history.loc[epoch] = np.mean([avg_loss, avg_acc, avg_val_loss, avg_val_acc], axis=1)
    liveloss.update(history.loc[epoch])
    liveloss.send()